In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import time
import numpy as np


class AxUpdater(object):
    
    def __init__(self, fig, ax, xlim=None, ylim=None):
        
        self.fig = fig
        self.ax = ax
        
        self.xlim = xlim
        if xlim is not None:
            self.ax.set_xlim(xlim[0],xlim[1])
        
        self.ylim = ylim
        if ylim is not None:
            self.ax.set_ylim(ylim[0],ylim[1])
        
        self.trajs = []
        self.background = self.fig.canvas.copy_from_bbox(self.ax.bbox)

    def new_plot(self):
        self.trajs.append(self.ax.plot([],[], '-o')[0])
        
        
    def update_plot(self,x=None,y=None):
        assert(len(self.trajs))
        if x is not None and y is not None:
            self.trajs[-1].set_data(x, y)
        
        self.fig.canvas.restore_region(self.background)
        # redraw just the points
        self.ax.draw_artist(self.trajs[-1])
        # fill in the axes rectangle
        self.fig.canvas.blit(self.ax.bbox)
        
    def clean(self):
        for idx in range(len(self.ax.lines)-1,-1,-1):
            if self.ax.lines[idx] not in self.trajs:
                self.ax.lines.pop(idx)
        for idx in range(len(self.ax.collections)-1,-1,-1):
            self.ax.collections.pop(idx)
        self.fig.canvas.draw()
        self.fig.canvas.flush_events()
        
    def clear(self):
        self.ax.set_prop_cycle(None)
        self.trajs=[]
        self.clean()

class DrawOnAx(AxUpdater):

    def __init__(self, fig, ax, xlim=(0.0,1.0),ylim=(0.0,1.0)):
        AxUpdater.__init__(self, fig, ax, xlim, ylim)
        
        self.curr_x = None
        self.curr_y = None
        self.curr_t = None
        self.curr_t_start = None
        self.data = []
        self.cbs = []

        self.do_record = False
        
        self.fig.canvas.mpl_connect("button_press_event", self.on_press)
        self.fig.canvas.mpl_connect("button_release_event", self.on_release)
        self.fig.canvas.mpl_connect("motion_notify_event", self.on_move)

    def add_cb(self,cb):
        assert(isinstance(cb,tuple) and len(cb)==3)
        
        self.cbs.append(cb)
        
    def on_press(self, event):
        if event.inaxes != self.ax:
            return
        if self.do_record:
            return
        self.curr_t_start = time.time() 
        self.curr_t = [0.0]
        self.curr_x = [event.xdata]
        self.curr_y = [event.ydata]
        self.new_plot()
        self.update_plot(self.curr_x,self.curr_y)
        self.do_record=True
        
        for cb in self.cbs:
            if cb[0] is not None:
                cb[0](self.curr_t,self.curr_x,self.curr_y)

    def on_move(self, event):
        if event.inaxes != self.ax:
            return
        if not self.do_record:
            return
        
        self.curr_t.append(time.time()-self.curr_t_start)
        self.curr_x.append(event.xdata)
        self.curr_y.append(event.ydata)
        self.update_plot(self.curr_x,self.curr_y)
        
        for cb in self.cbs:
            if cb[1] is not None:
                cb[1](self.curr_t,self.curr_x,self.curr_y)

    def on_release(self, event):
        if event.inaxes != self.ax:
            return
        if not self.do_record:
            return
        self.do_record = False
        self.curr_t.append(time.time()-self.curr_t_start)
        self.curr_x.append(event.xdata)
        self.curr_y.append(event.ydata)
        self.update_plot(self.curr_x,self.curr_y)
        self.data.append(np.array([self.curr_t, self.curr_x, self.curr_y]))
        
        for cb in self.cbs:
            if cb[2] is not None:
                cb[2](self.curr_t,self.curr_x,self.curr_y)


    def clear(self):
        self.data = []
        AxUpdater.clear(self)



# Motivation
$\newcommand\traj{\boldsymbol{\tau}}$
Given:
- Trajectory $\traj \in \mathbb{R}^{D\times L}$, with dimension $D$, over $L$ time steps

Desired: Movement representation that
- is parametrized
- is time invariant
- considers multiple demonstrations
- captures correlations between DoFs





In [2]:
fig, axs = plt.subplots(3,1,squeeze=False)

doa = DrawOnAx(fig, axs[0][0])
uax = AxUpdater(fig, axs[1][0],xlim=(0.0,3.0),ylim=(0.0,1.0))
uay = AxUpdater(fig, axs[2][0],xlim=(0.0,3.0),ylim=(0.0,1.0))

doa.add_cb((lambda t,x,y: (uax.new_plot(),uax.update_plot(t,x)), lambda t,x,y: uax.update_plot(t,x), lambda t,x,y: uax.update_plot(t,x)))
doa.add_cb((lambda t,x,y: (uay.new_plot(),uay.update_plot(t,y)), lambda t,x,y: uay.update_plot(t,y), lambda t,x,y: uay.update_plot(t,y)))


## Parametrized
$\newcommand\basis{\boldsymbol{\phi}}$
$\newcommand\w{\boldsymbol{w}}$
$\newcommand\phase{\boldsymbol{z}}$
- Considering each time step individually is often infeasible
    - large parameter space
    - how to ensure smoothness?
    - how to achieve time invariance?
    
- Parameterizing the trajectory using a fixed number $B$ of basis functions
    - $\traj = \basis(\phase)\w$
        - $\phase$: phase, i.e. (time)steps at which to compute $\tau$
        - $\basis(\phase)$: basis, e.g., normalized radial: $\phi_i(z_t) = \dfrac{b_i(z_t)}{\sum_j b_j(z_t)}$, $b_i=\exp\left(-\dfrac{(z_t-c_i)^2}{2h}\right)$
        - $\w$: weights


In [3]:
# Radial basis function
def radial_basis(phase,centers,bandwidth):
    bases = np.exp(-(np.repeat(phase[...,np.newaxis],centers.shape,-1)-centers)**2/(2*bandwidth)).T
    print(bases.shape)
    print(bases)
    bases /= bases.sum(axis=0)
    return bases.T
    
# The recorded data
data = doa.data[-1]

# interpolated data...
# usually not required since recorded data
# comes in certain frequency!!    
hz = 100
def interp_data(data,hz):
    t = np.arange(data[0,0],data[0,-1],1/hz)
    x = np.interp(t,data[0,:],data[1,:])
    y = np.interp(t,data[0,:],data[2,:])
    print("data is: " , data)
    print("t is: ", t)
    print("x is: " , x)
    print("y is: " , y)
    return np.array([t,x,y]) 



# Number of bases
B = 15

# The timesteps for the basis functions (phase)
z = np.linspace(data[0,0],data[0,-1],100)

# Equidistant centers for the bases
c = np.linspace(z[0],z[-1],B)

# A heursitic for a possible/plausible bandwidth (bandwidth)
h = -(c[1]-c[0])**2/(2*np.log(0.3))



#basis functions
ϕ = radial_basis(z,c,h)
print("ϕ.shape: {}".format(ϕ.shape))
w = np.ones(B)


doa.clean()
doa.ax.plot(data[1,:],data[2,:],'b-o',linewidth=3)

uax.clean()
uax.ax.plot(data[0,:],data[1,:],'b-o',linewidth=3)
uax.ax.plot(z,ϕ*w,linewidth=3)
uax.ax.plot(z,np.dot(ϕ,w),'g',linewidth=3)


uay.clean()
uay.ax.plot(data[0,:],data[2,:],'b-o',linewidth=3)
uay.ax.plot(z,ϕ*w,linewidth=3)
uay.ax.plot(z,np.dot(ϕ,w),'g',linewidth=3)


(15, 100)
[[1.00000000e+000 9.76210539e-001 9.08184255e-001 ... 4.11829097e-099
  3.76441924e-101 3.27918505e-103]
 [3.00000000e-001 4.11670403e-001 5.38350373e-001 ... 2.73321223e-085
  3.51187629e-087 4.30023359e-089]
 [8.10000000e-003 1.56242186e-002 2.87209352e-002 ... 1.63257143e-072
  2.94864809e-074 5.07528786e-076]
 ...
 [5.07528786e-076 2.94864809e-074 1.63257143e-072 ... 2.87209352e-002
  1.56242186e-002 8.10000000e-003]
 [4.30023359e-089 3.51187629e-087 2.73321223e-085 ... 5.38350373e-001
  4.11670403e-001 3.00000000e-001]
 [3.27918505e-103 3.76441924e-101 4.11829097e-099 ... 9.08184255e-001
  9.76210539e-001 1.00000000e+000]]
ϕ.shape: (100, 15)


<span style="color:#AA0000; font-weight:bold;">Obviously, something is wrong!</span>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

## Learning the weights $\w$

- straight forward approach learn the weights via ridge regression
    -- $\w = \left(\basis(\phase)^T\basis(\phase) + \lambda I\right)^{-1} \basis(\phase)^T\traj$


In [4]:

z = data[0,:]
ϕ = radial_basis(z,c,h)
    

# ridge regression
def learn_weights_1(data, ϕ, λ=1e-6):
    wx = np.dot(np.linalg.inv(np.dot(ϕ.T,ϕ)+λ*np.eye(B)),np.dot(ϕ.T,data[1,:]))
    wy = np.dot(np.linalg.inv(np.dot(ϕ.T,ϕ)+λ*np.eye(B)),np.dot(ϕ.T,data[2,:]))
    return np.array([wx,wy])

w1 = learn_weights_1(data, ϕ)
# also ridge regression but numerically more stable
def learn_weights_2(data, ϕ, λ=1e-6):
    wx = np.linalg.solve(np.dot(ϕ.T,ϕ)+λ*np.eye(B),np.dot(ϕ.T,data[1,:]))
    wy = np.linalg.solve(np.dot(ϕ.T,ϕ)+λ*np.eye(B),np.dot(ϕ.T,data[2,:]))
    return np.array([wx,wy])

w2 = learn_weights_2(data, ϕ)

print("max abs difference between inv and solve: {}".format(np.max(np.abs(w1-w2))))

# still ridge regression but for all dimensions at once
def learn_weights(data, ϕ, λ=1e-6):
    w = np.linalg.solve(np.dot(ϕ.T,ϕ)+λ*np.eye(B),np.dot(ϕ.T,data[1:,:].T)).T
    return w

w = learn_weights(data, ϕ)

print("w.shape: {}".format(w.shape))
print("max abs difference between solve and solve for all dims: {}".format(np.max(np.abs(w-w2))))

uax.clean()
uax.ax.plot(data[0,:],data[1,:],'r-x',linewidth=3)
uax.ax.plot(z,ϕ*w[0,:],linewidth=3)
uax.ax.plot(z,np.dot(ϕ,w[0,:]),'g-s',linewidth=3)


uay.clean()
uay.ax.plot(data[0,:],data[2,:],'r-x',linewidth=3)
uay.ax.plot(z,ϕ*w[1,:],linewidth=3)
uay.ax.plot(z,np.dot(ϕ,w[1,:]),'g-s',linewidth=3)


doa.clean()
doa.ax.plot(data[1,:],data[2,:],'r-x',linewidth=3)
doa.ax.plot(np.dot(ϕ,w[0,:]),np.dot(ϕ,w[1,:]),'g-s',linewidth=3)

(15, 22)
[[1.00000000e+000 7.17232685e-001 2.62108882e-003 9.83391360e-005
  4.69139829e-006 2.53319932e-008 1.92756396e-010 1.48587025e-013
  6.03843870e-016 7.61716000e-020 4.63708710e-024 8.78957998e-028
  2.70325297e-032 3.85308551e-036 1.33757928e-042 1.41969536e-047
  4.58093725e-054 2.40497262e-059 9.92838816e-074 7.83891105e-080
  2.84331234e-096 3.27918505e-103]
 [3.00000000e-001 7.62475163e-001 1.65668127e-001 2.31681011e-002
  3.06791981e-003 7.35776440e-005 1.86121141e-006 6.74326012e-009
  7.94211245e-011 4.81242572e-014 1.34670843e-017 8.78203862e-021
  1.08291917e-024 4.66963451e-028 9.15859815e-034 3.37246080e-038
  5.01469536e-044 8.52752621e-049 6.77115966e-062 1.71453707e-067
  1.16759193e-082 4.30023359e-089]
 [8.10000000e-003 7.29514352e-002 9.42407418e-001 4.91243707e-001
  1.80562771e-001 1.92337914e-002 1.61742862e-003 2.75423788e-005
  9.40134328e-007 2.73638694e-009 3.52001421e-012 7.89705336e-015
  3.90434244e-018 5.09330452e-021 5.64392176e-026 7.21009802e-0

<span style='color:green; font-weight:bold;'>Because of the parametrization we can reproduce the trajectory in a different resolution</span>

In [5]:
z = np.arange(data[0,0],data[0,-1],1/10)

ϕ = radial_basis(z,c,h)

uax.clean()
uax.ax.plot(z,ϕ*w[0,:],linewidth=3)
uax.ax.plot(z,np.dot(ϕ,w[0,:]),'g-s',linewidth=3)


uay.clean()
uay.ax.plot(z,ϕ*w[1,:],linewidth=3)
uay.ax.plot(z,np.dot(ϕ,w[1,:]),'g-s',linewidth=3)


doa.clean()
doa.ax.plot(np.dot(ϕ,w[0,:]),np.dot(ϕ,w[1,:]),'g-s',linewidth=3)

(15, 5)
[[1.00000000e+000 8.03334734e-007 4.16472356e-025 1.39337786e-055
  3.00846331e-098]
 [3.00000000e-001 8.96297063e-004 1.72812750e-018 2.15026844e-045
  1.72664346e-084]
 [8.10000000e-003 9.00015339e-002 6.45368692e-013 2.98647556e-036
  8.91873227e-072]
 [1.96830000e-005 8.13374135e-001 2.16911468e-008 3.73308397e-028
  4.14616036e-060]
 [4.30467210e-009 6.61566209e-001 6.56144728e-005 4.19970769e-021
  1.73472873e-049]
 [8.47288609e-014 4.84282506e-002 1.78632009e-002 4.25219210e-015
  6.53220129e-040]
 [1.50094635e-019 3.19055882e-004 4.37684768e-001 3.87479918e-010
  2.21375755e-031]
 [2.39299329e-026 1.89180880e-007 9.65175059e-001 3.17781076e-006
  6.75216522e-024]
 [3.43368382e-034 1.00955558e-011 1.91554897e-001 2.34557526e-003
  1.85352554e-017]
 [4.43426488e-043 4.84870475e-017 3.42155036e-003 1.55816420e-001
  4.57927368e-012]
 [5.15377521e-053 2.09586716e-023 5.50041076e-006 9.31578771e-001
  1.01820947e-007]
 [5.39103090e-064 8.15350375e-031 7.95810782e-010 5.01266

## Time Invariance

- How can we execute a learned trajectory faster or slower?
    1. extend phase $z$
    2. recompute centers $c$
    3. set new bandwidth $h$
    4. recompute basis

In [6]:
des_duration = 2

z = np.arange(data[0,0],des_duration,1/hz)
c = np.linspace(z[0],z[-1],B)
h = -(c[1]-c[0])**2/(2*np.log(0.3))

ϕ = radial_basis(z,c,h)

uax.clean()
uax.ax.plot(z,ϕ*w[0,:],linewidth=3)
uax.ax.plot(z,np.dot(ϕ,w[0,:]),'g-s',linewidth=3)


uay.clean()
uay.ax.plot(z,ϕ*w[1,:],linewidth=3)
uay.ax.plot(z,np.dot(ϕ,w[1,:]),'g-s',linewidth=3)


doa.clean()
doa.ax.plot(np.dot(ϕ,w[0,:]),np.dot(ϕ,w[1,:]),'g-s',linewidth=3)

(15, 200)
[[1.00000000e+000 9.94058812e-001 9.76446198e-001 ... 3.67619761e-101
  3.49277256e-102 3.27918505e-103]
 [3.00000000e-001 3.53267930e-001 4.11065786e-001 ... 3.43544669e-087
  3.86656710e-088 4.30023359e-089]
 [8.10000000e-003 1.12989700e-002 1.55745982e-002 ... 2.88941610e-074
  3.85232841e-075 5.07528786e-076]
 ...
 [5.07528786e-076 3.85232841e-075 2.88941610e-074 ... 1.55745982e-002
  1.12989700e-002 8.10000000e-003]
 [4.30023359e-089 3.86656710e-088 3.43544669e-087 ... 4.11065786e-001
  3.53267930e-001 3.00000000e-001]
 [3.27918505e-103 3.49277256e-102 3.67619761e-101 ... 9.76446198e-001
  9.94058812e-001 1.00000000e+000]]


### Simpler alternative:
- temporal scaling of each trajectory, such that it was always executed between $0.0$ and $1.0$
    - centers never have to change
    - bandwidth never has to change
    - phase is just normalized time



In [7]:
def get_phase(t):
    phase = t-np.min(t)
    phase /= np.max(phase)
    return phase

# Equidistant centers for the bases
c = np.linspace(0.0,1.0,B)
# A heursitic for a possible/plausible bandwidth
h = -(1/(B-1))**2/(2*np.log(0.3))

z = get_phase(data[0,:])
ϕ = radial_basis(z,c,h)

w = learn_weights(data,ϕ)

uax.clean()
uax.ax.plot(data[0,:],data[1,:],'r-x',linewidth=3)
uax.ax.plot(data[0,:],ϕ*w[0,:],linewidth=3)
uax.ax.plot(data[0,:],np.dot(ϕ,w[0,:]),'g-s',linewidth=3)


uay.clean()
uay.ax.plot(data[0,:],data[2,:],'r-x',linewidth=3)
uay.ax.plot(data[0,:],ϕ*w[1,:],linewidth=3)
uay.ax.plot(data[0,:],np.dot(ϕ,w[1,:]),'g-s',linewidth=3)


doa.clean()
doa.ax.plot(data[1,:],data[2,:],'r-x',linewidth=3)
doa.ax.plot(np.dot(ϕ,w[0,:]),np.dot(ϕ,w[1,:]),'g-s',linewidth=3)

(15, 22)
[[1.00000000e+000 7.17232685e-001 2.62108882e-003 9.83391360e-005
  4.69139829e-006 2.53319932e-008 1.92756396e-010 1.48587025e-013
  6.03843870e-016 7.61716000e-020 4.63708710e-024 8.78957998e-028
  2.70325297e-032 3.85308551e-036 1.33757928e-042 1.41969536e-047
  4.58093725e-054 2.40497262e-059 9.92838816e-074 7.83891105e-080
  2.84331234e-096 3.27918505e-103]
 [3.00000000e-001 7.62475163e-001 1.65668127e-001 2.31681011e-002
  3.06791981e-003 7.35776440e-005 1.86121141e-006 6.74326012e-009
  7.94211245e-011 4.81242572e-014 1.34670843e-017 8.78203862e-021
  1.08291917e-024 4.66963451e-028 9.15859815e-034 3.37246080e-038
  5.01469536e-044 8.52752621e-049 6.77115966e-062 1.71453707e-067
  1.16759193e-082 4.30023359e-089]
 [8.10000000e-003 7.29514352e-002 9.42407418e-001 4.91243707e-001
  1.80562771e-001 1.92337914e-002 1.61742862e-003 2.75423788e-005
  9.40134328e-007 2.73638694e-009 3.52001421e-012 7.89705336e-015
  3.90434244e-018 5.09330452e-021 5.64392176e-026 7.21009802e-0

reproductions in different velocities are now simpler
    - compute phase of desired time axis
    - recompute basis

In [8]:

des_t = np.arange(0.0,des_duration,1/hz)

z = get_phase(des_t)
ϕ = radial_basis(z,c,h)


uax.clean()
uax.ax.plot(data[0,:],data[1,:],'r-x',linewidth=3)
uax.ax.plot(des_t,ϕ*w[0,:],linewidth=3)
uax.ax.plot(des_t,np.dot(ϕ,w[0,:]),'g-s',linewidth=3)


uay.clean()
uay.ax.plot(data[0,:],data[2,:],'r-x',linewidth=3)
uay.ax.plot(des_t,ϕ*w[1,:],linewidth=3)
uay.ax.plot(des_t,np.dot(ϕ,w[1,:]),'g-s',linewidth=3)


doa.clean()
doa.ax.plot(data[1,:],data[2,:],'r-x',linewidth=3)
doa.ax.plot(np.dot(ϕ,w[0,:]),np.dot(ϕ,w[1,:]),'g-s',linewidth=3)

interp_data(data, hz)

(15, 200)
[[1.00000000e+000 9.94058812e-001 9.76446198e-001 ... 3.67619761e-101
  3.49277256e-102 3.27918505e-103]
 [3.00000000e-001 3.53267930e-001 4.11065786e-001 ... 3.43544669e-087
  3.86656710e-088 4.30023359e-089]
 [8.10000000e-003 1.12989700e-002 1.55745982e-002 ... 2.88941610e-074
  3.85232841e-075 5.07528786e-076]
 ...
 [5.07528786e-076 3.85232841e-075 2.88941610e-074 ... 1.55745982e-002
  1.12989700e-002 8.10000000e-003]
 [4.30023359e-089 3.86656710e-088 3.43544669e-087 ... 4.11065786e-001
  3.53267930e-001 3.00000000e-001]
 [3.27918505e-103 3.49277256e-102 3.67619761e-101 ... 9.76446198e-001
  9.94058812e-001 1.00000000e+000]]
data is:  [[0.         0.01538873 0.06507993 0.08108377 0.09350181 0.11163783
  0.12624979 0.14507389 0.15801692 0.1771059  0.19565988 0.21068883
  0.22757983 0.241045   0.26210785 0.27723885 0.2958231  0.31011891
  0.34608293 0.36025786 0.39592791 0.41005111]
 [0.09475806 0.09677419 0.09677419 0.10282258 0.10887097 0.13104839
  0.15725806 0.19354839 0

array([[0.        , 0.01      , 0.02      , 0.03      , 0.04      ,
        0.05      , 0.06      , 0.07      , 0.08      , 0.09      ,
        0.1       , 0.11      , 0.12      , 0.13      , 0.14      ,
        0.15      , 0.16      , 0.17      , 0.18      , 0.19      ,
        0.2       , 0.21      , 0.22      , 0.23      , 0.24      ,
        0.25      , 0.26      , 0.27      , 0.28      , 0.29      ,
        0.3       , 0.31      , 0.32      , 0.33      , 0.34      ,
        0.35      , 0.36      , 0.37      , 0.38      , 0.39      ,
        0.4       , 0.41      ],
       [0.09475806, 0.0960682 , 0.09677419, 0.09677419, 0.09677419,
        0.09677419, 0.09677419, 0.09863365, 0.10241299, 0.10716536,
        0.11681721, 0.12904559, 0.14604773, 0.16448796, 0.18376661,
        0.19815241, 0.21046246, 0.23475446, 0.25610438, 0.27023056,
        0.28754143, 0.30900537, 0.33048892, 0.35112263, 0.36909014,
        0.39153982, 0.41451254, 0.43933504, 0.46335267, 0.48396499,
        0.50892

Defining a normalized phase also makes it easier to combine multiple demonstrations


## Considering Multiple Demonstrations

- We consider each demonstration as a 'variation' or 'sample' of the same movement primitive
- hence, we treat the weight vector for each demonstration $\w_i$ as an instance of a random variable, drawn from a multivariate Gaussian $\w_i \sim \mathcal{N}(\w|\mu_w,\Sigma_w)$

In [9]:
doa.clear()
uax.clear()
uay.clear()



In [10]:
hz = 100
B = 15
D = 2

# Equidistant centers for the bases
c = np.linspace(0.0,1.0,B)
# A heursitic for a possible/plausible bandwidth
h = -(1/(B-1))**2/(2*np.log(0.3))

trajectories = [interp_data(d,hz) for d in doa.data]

def learn_weight_distribution(trajectories):
    ws = np.array([learn_weights(d,radial_basis(get_phase(d[0,:]),c,h)).flatten() for d in trajectories])
    μ = np.mean(ws,axis=0)
    Σ = np.cov(ws.T)
    return μ, Σ

μ_w, Σ_w = learn_weight_distribution(trajectories)


data is:  [[0.         0.13211513 0.16460323 0.17911601 0.19795394 0.23166013
  0.26532125 0.27928209 0.29698205 0.31234026 0.33189011 0.34589005
  0.36441898 0.3792541  0.39664412 0.40961313 0.43130803 0.46418929
  0.49950218 0.51348186 0.52613807 0.54697704 0.58207989 0.614048
  0.62823391 0.64771414 0.66202426 0.67986703 0.69331193 0.71370316
  0.72910905 0.76622105 0.78080416 0.81508708 0.84821129 0.8818872 ]
 [0.15322581 0.15322581 0.16935484 0.18346774 0.19556452 0.21572581
  0.23991935 0.25201613 0.27016129 0.28830645 0.31653226 0.32056452
  0.34677419 0.35887097 0.37298387 0.37903226 0.39919355 0.42540323
  0.46169355 0.48387097 0.49395161 0.52620968 0.57862903 0.62096774
  0.63508065 0.64717742 0.66330645 0.6733871  0.68145161 0.69153226
  0.70362903 0.72379032 0.72983871 0.73790323 0.74798387 0.74798387]
 [0.23279221 0.24199134 0.35238095 0.41677489 0.4719697  0.5547619
  0.60075758 0.61915584 0.62835498 0.63755411 0.64675325 0.64675325
  0.64675325 0.64675325 0.62835498 0.61

We can now sample from the weight distribution and produce new similar trajectories

In [11]:
doa.clean()
uax.clean()
uay.clean()

des_duration = 2
des_t = np.arange(0.0,des_duration,1/hz)

z = get_phase(des_t)
ϕ = radial_basis(z,c,h)

ws = np.random.multivariate_normal(μ_w, Σ_w, 10)
ws = ws.reshape(ws.shape[0],-1,B)
print(ϕ.shape)
uax.ax.plot(des_t,np.dot(ϕ,ws[:,0,:].T),linewidth=3)
uay.ax.plot(des_t,np.dot(ϕ,ws[:,1,:].T),linewidth=3)
doa.ax.plot(np.dot(ϕ,ws[:,0,:].T),np.dot(ϕ,ws[:,1,:].T),linewidth=3)

(15, 200)
[[1.00000000e+000 9.94058812e-001 9.76446198e-001 ... 3.67619761e-101
  3.49277256e-102 3.27918505e-103]
 [3.00000000e-001 3.53267930e-001 4.11065786e-001 ... 3.43544669e-087
  3.86656710e-088 4.30023359e-089]
 [8.10000000e-003 1.12989700e-002 1.55745982e-002 ... 2.88941610e-074
  3.85232841e-075 5.07528786e-076]
 ...
 [5.07528786e-076 3.85232841e-075 2.88941610e-074 ... 1.55745982e-002
  1.12989700e-002 8.10000000e-003]
 [4.30023359e-089 3.86656710e-088 3.43544669e-087 ... 4.11065786e-001
  3.53267930e-001 3.00000000e-001]
 [3.27918505e-103 3.49277256e-102 3.67619761e-101 ... 9.76446198e-001
  9.94058812e-001 1.00000000e+000]]
(200, 15)


We can even write down the distribution in the original trajectory space:
- $\traj \sim \mathcal{N}(\traj|\mu_\tau,\Sigma_\tau)$
    - $\mu_\tau = \Psi\mu_w$
    - $\Sigma_\tau = \Psi\Sigma_w\Psi^T+\Sigma_{\mathrm{obs}}$
    - $\Psi$: block diagonal of $D$ blocks. Each block being $\phi$

In [12]:


def get_traj_distribution(μ_w, Σ_w, des_duration=1.0):
    des_t = np.arange(0.0,des_duration,1/hz)
    z = get_phase(des_t)
    ϕ = radial_basis(z,c,h)
    D = 2
    Ψ = np.kron(np.eye(int(μ_w.shape[0]/B),dtype=int),ϕ)
    
    μ = np.dot(Ψ,μ_w)
    Σ = np.dot(np.dot(Ψ,Σ_w),Ψ.T)
    return μ, Σ


μ_τ, Σ_τ = get_traj_distribution(μ_w, Σ_w, des_duration)


des_t = np.arange(0.0,des_duration,1/hz)
μ_D = μ_τ.reshape((-1,des_t.shape[0]))


uax.ax.plot(des_t,μ_D[0,:],'m',linewidth=5)
uay.ax.plot(des_t,μ_D[1,:],'m',linewidth=5)


σ_τ = np.sqrt(np.diag(Σ_τ))
σ_D = σ_τ.reshape((-1,des_t.shape[0]))

uax.ax.fill_between(des_t, μ_D[0,:]-2*σ_D[0,:], μ_D[0,:]+2*σ_D[0,:], color='m',alpha=0.3)
uay.ax.fill_between(des_t, μ_D[1,:]-2*σ_D[1,:], μ_D[1,:]+2*σ_D[1,:], color='m',alpha=0.3)


(15, 200)
[[1.00000000e+000 9.94058812e-001 9.76446198e-001 ... 3.67619761e-101
  3.49277256e-102 3.27918505e-103]
 [3.00000000e-001 3.53267930e-001 4.11065786e-001 ... 3.43544669e-087
  3.86656710e-088 4.30023359e-089]
 [8.10000000e-003 1.12989700e-002 1.55745982e-002 ... 2.88941610e-074
  3.85232841e-075 5.07528786e-076]
 ...
 [5.07528786e-076 3.85232841e-075 2.88941610e-074 ... 1.55745982e-002
  1.12989700e-002 8.10000000e-003]
 [4.30023359e-089 3.86656710e-088 3.43544669e-087 ... 4.11065786e-001
  3.53267930e-001 3.00000000e-001]
 [3.27918505e-103 3.49277256e-102 3.67619761e-101 ... 9.76446198e-001
  9.94058812e-001 1.00000000e+000]]


In [13]:
uax.clean()
uay.clean()

uax.ax.fill_between(des_t, μ_D[0,:]-2*σ_D[0,:], μ_D[0,:]+2*σ_D[0,:], color='m',alpha=0.3)
uay.ax.fill_between(des_t, μ_D[1,:]-2*σ_D[1,:], μ_D[1,:]+2*σ_D[1,:], color='m',alpha=0.3)


#implemented function
def conditioning(μ_w, Σ_w, y_t, Σ_y, Ψ):
    
    inverse = np.linalg.inv(Σ_y + np.dot(np.dot(Ψ.T, Σ_w), Ψ))
    L = np.dot(np.dot(Σ_w, Ψ), inverse)
    
    new_μ_w = μ_w + np.dot(L, y_t - np.dot(Ψ.T, μ_w))
    new_Σ_w = Σ_w - np.dot(np.dot(L, Ψ.T), Σ_w)
    return new_μ_w, new_Σ_w

#gets Ψ based on dimensions and timestep conditioned on
def get_Ψ(z, dims, D, B):
    z = np.array(z,ndmin=1)
    length = z.shape[0]
    c = np.linspace(0.0,1.0,B)
    h = -(1/(B-1))**2/(2*np.log(0.3))
    
    basis = radial_basis(z,c,h)
    Ψ = np.zeros((D * B, len(dims) * length))
    for idx, d in enumerate(dims):
        Ψ[d * B : (d+1) * B, idx * length : (idx + 1) * length] = basis.T
    return Ψ
    
    
#which dimensions conditioned on
dims = [0, 1]
#put value between 0 and 1 for timestep conditioned on
z = [.25]
#put values at x, y, z
features = [.4, .7]
#how much slack you give the distribution
Σ = .001

#c_Ψ = R. D* times #L X D time B
c_Ψ = get_Ψ(z, dims, D, B)

#y* is value of x and y we want to condition to in vector form
#Σ* is a D X D matrix (covariance matrix)
y_t = np.array(features)
Σ_y = np.diag([Σ]*len(y_t))

#call to function
c_μ_τ, c_Σ_τ = conditioning(μ_w, Σ_w, y_t, Σ_y, c_Ψ)

#get new mean with conditioning
new_μ_τ, new_Σ_τ = get_traj_distribution(c_μ_τ, c_Σ_τ, des_duration)

des_t = np.arange(0.0,des_duration,1/hz)
μ_D = new_μ_τ.reshape((-1,des_t.shape[0]))


uax.ax.plot(des_t,μ_D[0,:],'b',linewidth=5)
uay.ax.plot(des_t,μ_D[1,:],'b',linewidth=5)


σ_τ = np.sqrt(np.diag(new_Σ_τ))
σ_D = σ_τ.reshape((-1,des_t.shape[0]))

uax.ax.fill_between(des_t, μ_D[0,:]-2*σ_D[0,:], μ_D[0,:]+2*σ_D[0,:], color='b',alpha=0.3)
uay.ax.fill_between(des_t, μ_D[1,:]-2*σ_D[1,:], μ_D[1,:]+2*σ_D[1,:], color='b',alpha=0.3)



(15, 1)
[[3.93310346e-07]
 [5.39520364e-04]
 [6.66074524e-02]
 [7.40082804e-01]
 [7.40082804e-01]
 [6.66074524e-02]
 [5.39520364e-04]
 [3.93310346e-07]
 [2.58050918e-11]
 [1.52376486e-16]
 [8.09791123e-23]
 [3.87320584e-30]
 [1.66728811e-38]
 [6.45941575e-48]
 [2.25225901e-58]]
(15, 200)
[[1.00000000e+000 9.94058812e-001 9.76446198e-001 ... 3.67619761e-101
  3.49277256e-102 3.27918505e-103]
 [3.00000000e-001 3.53267930e-001 4.11065786e-001 ... 3.43544669e-087
  3.86656710e-088 4.30023359e-089]
 [8.10000000e-003 1.12989700e-002 1.55745982e-002 ... 2.88941610e-074
  3.85232841e-075 5.07528786e-076]
 ...
 [5.07528786e-076 3.85232841e-075 2.88941610e-074 ... 1.55745982e-002
  1.12989700e-002 8.10000000e-003]
 [4.30023359e-089 3.86656710e-088 3.43544669e-087 ... 4.11065786e-001
  3.53267930e-001 3.00000000e-001]
 [3.27918505e-103 3.49277256e-102 3.67619761e-101 ... 9.76446198e-001
  9.94058812e-001 1.00000000e+000]]
